In [2]:
pip install opencv-python imutils scipy numpy matplotlib tensorflow==2.12 mediapipe fer opencv-python dlib

<class 'OSError'>: Not available

In [ ]:
import cv2
import dlib

# Teste OpenCV
print(f"OpenCV version: {cv2.__version__}")

# Teste Dlib
detector = dlib.get_frontal_face_detector()
print("Dlib está funcionando corretamente.")


In [ ]:
import requests
import bz2
import shutil

url = "https://github.com/davisking/dlib-models/raw/master/shape_predictor_68_face_landmarks.dat.bz2"
output_file = "shape_predictor_68_face_landmarks.dat.bz2"

# Baixa o arquivo
response = requests.get(url)
with open(output_file, 'wb') as file:
    file.write(response.content)

# Descompacta o arquivoA
with bz2.BZ2File(output_file, "rb") as source, open("shape_predictor_68_face_landmarks.dat", "wb") as dest:
    shutil.copyfileobj(source, dest)

print("Download e descompactação concluídos.")

In [ ]:
import requests

# URL do arquivo no GitHub
url = "https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5"
output_file = "fer2013_mini_XCEPTION.102-0.66.hdf5"

# Faz o download do arquivo
response = requests.get(url, stream=True)
with open(output_file, 'wb') as file:
    # Escreve o conteúdo em partes para não sobrecarregar a memória
    shutil.copyfileobj(response.raw, file)

print(f"Download do arquivo {output_file} concluído.")


In [ ]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
import time
from datetime import datetime
import pandas as pd
from scipy.spatial import distance

class EnhancedNeuroAssessmentToolkit:
    def __init__(self):
        self.setup_detectors()
        self.setup_models()
        self.setup_parameters()

    def setup_detectors(self):
        """Configura detectores faciais e outros sensores"""
        self.face_detector = dlib.get_frontal_face_detector()
        self.landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    def setup_models(self):
        """Configura modelos especializados para diferentes aspectos"""
        base_model = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )

        # Modelo de classificação para expressões faciais
        self.expression_model = Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dense(128, activation='relu'),
            Dense(7, activation='softmax')  # 7 expressões básicas
        ])

    def setup_parameters(self):
        """Configura parâmetros de análise"""
        self.data_buffers = {
            'gaze': [],
            'head_movements': [],
            'expressions': [],
            'eye_aspect_ratio': []
        }
        self.max_buffer_size = 300  # 10 segundos a 30 fps

    def calculate_eye_aspect_ratio(self, eye_landmarks):
        """Calcula razão de aspecto dos olhos para detecção de piscadas"""
        v1 = distance.euclidean(eye_landmarks[1], eye_landmarks[5])
        v2 = distance.euclidean(eye_landmarks[2], eye_landmarks[4])
        h = distance.euclidean(eye_landmarks[0], eye_landmarks[3])
        return (v1 + v2) / (2.0 * h)

    def analyze_attention(self, eye_aspect_ratio):
        """Analisa atenção com base na razão de aspecto dos olhos (EAR)"""
        # Defina um limite de EAR para considerar que a pessoa está atenta
        EAR_THRESHOLD = 0.2
        if eye_aspect_ratio < EAR_THRESHOLD:
            return "Potencialmente distraído"
        else:
            return "Atenção mantida"

    def analyze_gaze(self, landmarks):
        """Analisa direção e estabilidade do olhar"""
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        # Calcular centro dos olhos
        left_center = np.mean(left_eye, axis=0)
        right_center = np.mean(right_eye, axis=0)

        # Calcular direção do olhar
        gaze_direction = np.array([
            right_center[0] - left_center[0],
            right_center[1] - left_center[1]
        ])

        return {
            'direction': gaze_direction,
            'stability': np.std(gaze_direction)
        }

    def analyze_head_movements(self, landmarks, prev_landmarks):
        """Analisa movimentos da cabeça"""
        if prev_landmarks is None:
            return {'movement': 0, 'stability': 1.0}

        movement = np.mean(np.abs(landmarks - prev_landmarks))
        stability = 1.0 / (1.0 + movement)

        return {
            'movement': movement,
            'stability': stability
        }

    def process_frame(self, frame, prev_landmarks=None):
        """Processa um frame com análises expandidas"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_detector(gray, 0)

        frame_data = {
            'timestamp': datetime.now(),
            'gaze_metrics': None,
            'head_movement': None,
            'expression': 'neutral',
            'eye_aspect_ratio': None,
            'attention': None
        }

        for face in faces:
            landmarks = self.landmark_predictor(gray, face)
            landmarks = face_utils.shape_to_np(landmarks)

            # Análises
            gaze_metrics = self.analyze_gaze(landmarks)
            head_metrics = self.analyze_head_movements(landmarks, prev_landmarks)
            ear = self.calculate_eye_aspect_ratio(landmarks[36:42])  # Olho esquerdo
            attention = self.analyze_attention(ear)

            # Atualizar dados do frame
            frame_data.update({
                'gaze_metrics': gaze_metrics,
                'head_movement': head_metrics,
                'eye_aspect_ratio': ear,
                'attention': attention
            })

            # Para análise de expressões faciais (se você tiver um método de predição)
            # expression_pred = self.expression_model.predict(frame)  # Pseudo código
            # frame_data['expression'] = expression_pred

            return frame, frame_data, landmarks

        return frame, frame_data, None

    def start_assessment(self, duration_seconds=300):
        """Inicia uma sessão de avaliação geral"""
        cap = cv2.VideoCapture(0)
        start_time = time.time()
        frames_data = []
        prev_landmarks = None

        try:
            while (time.time() - start_time) < duration_seconds:
                ret, frame = cap.read()
                if not ret:
                    break

                processed_frame, frame_data, landmarks = self.process_frame(frame, prev_landmarks)

                if frame_data:
                    frames_data.append(frame_data)

                prev_landmarks = landmarks

                cv2.imshow('Assessment', processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            cap.release()
            cv2.destroyAllWindows()

        return self.generate_report(frames_data)

    def generate_report(self, frames_data):
        """Gera relatório detalhado baseado na avaliação"""
        df = pd.DataFrame(frames_data)

        report = {
            'timestamp': datetime.now(),
            'average_attention': df['attention'].mode()[0] if not df['attention'].isnull().all() else 'Indeterminado',
            'average_gaze_direction': df['gaze_metrics'].apply(lambda x: x['direction'] if x is not None else np.nan).mean(),
            'average_head_movement': df['head_movement'].apply(lambda x: x['movement'] if x is not None else np.nan).mean(),
            'average_eye_aspect_ratio': df['eye_aspect_ratio'].mean(),
            'average_expression': df['expression'].mode()[0] if not df['expression'].isnull().all() else 'Indeterminado'
        }

        return report

# Uso do toolkit
toolkit = EnhancedNeuroAssessmentToolkit()
report = toolkit.start_assessment(duration_seconds=60)
print(report)


In [ ]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
import time
from datetime import datetime
import matplotlib.pyplot as plt

class NeuroAssessmentToolkit:
    def __init__(self):
        """Inicializa o toolkit com suporte a câmera"""
        self.setup_detectors()
        self.setup_models()
        self.setup_parameters()
        
    def setup_detectors(self):
        """Configura detectores faciais"""
        try:
            self.face_detector = dlib.get_frontal_face_detector()
            self.landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
        except Exception as e:
            print(f"Erro configurando detectores: {e}")
            raise
            
    def setup_models(self):
        """Configura modelos de deep learning"""
        try:
            # Modelo de atenção
            base_model = EfficientNetB0(
                weights='imagenet',
                include_top=False,
                input_shape=(224, 224, 3)
            )
            
            self.attention_model = Sequential([
                base_model,
                GlobalAveragePooling2D(),
                Dense(256, activation='relu'),
                Dense(3, activation='softmax')  # Três classes de atenção
            ])
            
            # Modelo de emoção (exemplo simplificado)
            self.emotion_model = Sequential([
                base_model,
                GlobalAveragePooling2D(),
                Dense(256, activation='relu'),
                Dense(7, activation='softmax')  # Exemplo: 7 emoções básicas
            ])
            
        except Exception as e:
            print(f"Erro configurando modelos: {e}")
            raise
            
    def setup_parameters(self):
        """Configura parâmetros de análise"""
        self.attention_buffer = []
        self.emotion_buffer = []
        self.head_movement_buffer = []
        self.gaze_direction_buffer = []
        self.eye_aspect_ratio_buffer = []
        self.frame_buffer = []
        self.max_buffer_size = 30  # 1 segundo a 30 fps
        
    def process_frame(self, frame):
        """Processa um frame de vídeo"""
        try:
            # Converter para escala de cinza para detecção facial
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detectar faces
            faces = self.face_detector(gray, 0)
            
            frame_data = {
                'timestamp': datetime.now(),
                'attention': 0,
                'emotion': 'neutral',
                'landmarks': None,
                'head_movement': 0,
                'gaze_direction': None,
                'eye_aspect_ratio': 0
            }
            
            for face in faces:
                # Extrair landmarks faciais
                landmarks = self.landmark_predictor(gray, face)
                landmarks = face_utils.shape_to_np(landmarks)
                frame_data['landmarks'] = landmarks
                
                # Analisar atenção
                attention_score = self.analyze_attention(frame, face)
                frame_data['attention'] = attention_score
                
                # Analisar emoção
                emotion = self.analyze_emotion(frame, face)
                frame_data['emotion'] = emotion
                
                # Analisar movimento da cabeça
                head_movement = self.analyze_head_movement(landmarks)
                frame_data['head_movement'] = head_movement
                
                # Analisar direção do olhar
                gaze_direction = self.analyze_gaze(landmarks)
                frame_data['gaze_direction'] = gaze_direction
                
                # Analisar proporção dos olhos
                eye_aspect_ratio = self.calculate_eye_aspect_ratio(landmarks)
                frame_data['eye_aspect_ratio'] = eye_aspect_ratio
                
                # Desenhar resultados no frame
                self.draw_analysis(frame, face, landmarks, attention_score, emotion)
                
            return frame, frame_data
            
        except Exception as e:
            print(f"Erro processando frame: {e}")
            return frame, None
            
    def analyze_attention(self, frame, face):
        """Analisa nível de atenção"""
        try:
            # Extrair ROI da face
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_img = frame[y:y+h, x:x+w]
            
            # Redimensionar para input do modelo
            face_img = cv2.resize(face_img, (224, 224))
            face_img = tf.keras.applications.efficientnet.preprocess_input(face_img)
            face_img = np.expand_dims(face_img, axis=0)
            
            # Predição
            attention_pred = self.attention_model.predict(face_img, verbose=0)
            
            return float(attention_pred[0][0])  # Sustained attention score
            
        except Exception as e:
            print(f"Erro analisando atenção: {e}")
            return 0
            
    def analyze_emotion(self, frame, face):
        """Analisa emoção"""
        try:
            # Extrair ROI da face
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_img = frame[y:y+h, x:x+w]
            
            # Redimensionar para input do modelo
            face_img = cv2.resize(face_img, (224, 224))
            face_img = tf.keras.applications.efficientnet.preprocess_input(face_img)
            face_img = np.expand_dims(face_img, axis=0)
            
            # Predição
            emotion_pred = self.emotion_model.predict(face_img, verbose=0)
            emotion_labels = ['neutral', 'happy', 'sad', 'angry', 'surprised', 'disgusted', 'fearful']
            return emotion_labels[np.argmax(emotion_pred)]
        
        except Exception as e:
            print(f"Erro analisando emoção: {e}")
            return 'neutral'
    
    def analyze_head_movement(self, landmarks):
        """Analisa o movimento da cabeça"""
        # Simples exemplo, implementar lógica para calcular movimento da cabeça
        # Você pode utilizar a diferença entre landmarks em frames sucessivos
        return np.random.rand()  # Substitua por cálculo real

    def analyze_gaze(self, landmarks):
        """Analisa a direção do olhar"""
        # Simples exemplo, retornar uma direção fixa
        return np.random.rand(2) * 100  # Substitua por cálculo real

    def calculate_eye_aspect_ratio(self, landmarks):
        """Calcula a proporção dos olhos"""
        left_eye = landmarks[36:42]  # Coordenadas dos pontos do olho esquerdo
        right_eye = landmarks[42:48]  # Coordenadas dos pontos do olho direito
        
        # Calcular a altura e largura dos olhos
        left_eye_aspect_ratio = (np.linalg.norm(left_eye[1] - left_eye[5]) +
                                  np.linalg.norm(left_eye[2] - left_eye[4])) / (2.0 * np.linalg.norm(left_eye[0] - left_eye[3]))
        right_eye_aspect_ratio = (np.linalg.norm(right_eye[1] - right_eye[5]) +
                                   np.linalg.norm(right_eye[2] - right_eye[4])) / (2.0 * np.linalg.norm(right_eye[0] - right_eye[3]))

        return (left_eye_aspect_ratio + right_eye_aspect_ratio) / 2.0
    
    def draw_analysis(self, frame, face, landmarks, attention_score, emotion):
        """Desenha resultados da análise no frame"""
        try:
            # Desenhar retângulo da face
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Desenhar landmarks
            for (x, y) in landmarks:
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
                
            # Mostrar scores
            cv2.putText(frame, f"Attention: {attention_score:.2f}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Emotion: {emotion}", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                       
        except Exception as e:
            print(f"Erro desenhando análise: {e}")
            
    def start_assessment(self, duration_seconds=30):
        """Inicia uma sessão de avaliação usando a câmera"""
        try:
            cap = cv2.VideoCapture(0)
            
            if not cap.isOpened():
                raise Exception("Não foi possível acessar a câmera")
                
            start_time = time.time()
            frames_data = []
            
            while (time.time() - start_time) < duration_seconds:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Processar frame
                processed_frame, frame_data = self.process_frame(frame)
                
                if frame_data:
                    frames_data.append(frame_data)
                
                # Mostrar frame
                cv2.imshow('Assessment', processed_frame)
                
                # Pressione 'q' para sair
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
            cap.release()
            cv2.destroyAllWindows()
            
            # Gerar relatório
            self.generate_report(frames_data)
            
        except Exception as e:
            print(f"Erro durante avaliação: {e}")
            if 'cap' in locals():
                cap.release()
            cv2.destroyAllWindows()
            
    def generate_report(self, frames_data):
        """Gera relatório da avaliação"""
        try:
            attention_scores = [f['attention'] for f in frames_data]
            gaze_directions = [f['gaze_direction'] for f in frames_data]
            head_movements = [f['head_movement'] for f in frames_data]
            eye_aspect_ratios = [f['eye_aspect_ratio'] for f in frames_data]
            emotions = [f['emotion'] for f in frames_data]
            
            plt.figure(figsize=(12, 6))
            plt.plot(attention_scores, label='Attention Levels', color='b')
            plt.title('Attention Levels During Assessment')
            plt.xlabel('Frame')
            plt.ylabel('Attention Score')
            plt.legend()
            plt.show()
            
            print("\nRelatório da Avaliação:")
            print(f"Duração: {len(frames_data)} frames")
            print(f"Atenção média: {np.mean(attention_scores):.2f}")
            print(f"Variação da atenção: {np.std(attention_scores):.2f}")
            print(f"Direção média do olhar: {np.mean(gaze_directions, axis=0)}")
            print(f"Média do movimento da cabeça: {np.mean(head_movements):.2f}")
            print(f"Proporção média dos olhos: {np.mean(eye_aspect_ratios):.2f}")
            print(f"Emoção mais comum: {max(set(emotions), key=emotions.count)}")
            
        except Exception as e:
            print(f"Erro gerando relatório: {e}")

# Uso do toolkit
toolkit = NeuroAssessmentToolkit()
toolkit.start_assessment(duration_seconds=30)


In [ ]:
import cv2
import dlib
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

class NeuroBehaviorToolkit:
    def __init__(self):
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        self.blink_counter = 0
        self.eye_movements = []
        self.body_movements = []
        self.emotions = []


    def start_assessment(self, duration_seconds=10):
        """Inicia a avaliação geral por vídeo."""
        cap = cv2.VideoCapture(0)
        start_time = datetime.now()

        while (datetime.now() - start_time).seconds < duration_seconds:
            ret, frame = cap.read()
            if not ret:
                break
            self.process_frame(frame)
            cv2.imshow("Avaliação Neuropsicológica", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        self.generate_report()

    def process_frame(self, frame):
        """Processa um frame para detectar piscadas, movimentos e expressões faciais."""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray)

        for face in faces:
            landmarks = self.predictor(gray, face)

            # Detecta piscadas
            if self.is_blinking(landmarks):
                self.blink_counter += 1

            # Rastreia movimentos oculares
            eye_center = self.get_eye_center(landmarks, [36, 39])
            self.eye_movements.append(eye_center)

            # Detecta expressões faciais (emocional)
            emotion = self.detect_emotion(landmarks)
            self.emotions.append(emotion)

        # Detecta movimentos corporais
        movement_intensity = self.detect_body_movement(frame)
        self.body_movements.append(movement_intensity)

    def is_blinking(self, landmarks):
        """Verifica se o olho está piscando."""
        left_ear = self.calculate_eye_aspect_ratio(landmarks, [36, 37, 38, 39, 40, 41])
        right_ear = self.calculate_eye_aspect_ratio(landmarks, [42, 43, 44, 45, 46, 47])
        return left_ear < 0.25 or right_ear < 0.25

    def detect_emotion(self, landmarks):
        """Simples detecção de expressão emocional (exemplo)."""
        mouth_aspect_ratio = self.calculate_mouth_aspect_ratio(landmarks, [48, 51, 57, 54])
        if mouth_aspect_ratio > 0.5:
            return "Euforia"
        elif mouth_aspect_ratio < 0.3:
            return "Tristeza"
        return "Neutro"

    def calculate_eye_aspect_ratio(self, landmarks, points):
        """Calcula a razão de aspecto do olho (EAR)."""
        p1, p2, p3, p4, p5, p6 = [landmarks.part(i) for i in points]
        horizontal_dist = np.linalg.norm([p4.x - p1.x, p4.y - p1.y])
        vertical_dist = (np.linalg.norm([p2.x - p6.x, p2.y - p6.y]) +
                         np.linalg.norm([p3.x - p5.x, p3.y - p5.y])) / 2.0
        return vertical_dist / horizontal_dist

    def calculate_mouth_aspect_ratio(self, landmarks, points):
        """Calcula a razão de aspecto da boca."""
        p1, p2, p3, p4 = [landmarks.part(i) for i in points]
        vertical_dist = np.linalg.norm([p2.x - p3.x, p2.y - p3.y])
        horizontal_dist = np.linalg.norm([p1.x - p4.x, p1.y - p4.y])
        return vertical_dist / horizontal_dist

    def get_eye_center(self, landmarks, points):
        """Calcula o centro do olho."""
        x = np.mean([landmarks.part(p).x for p in points])
        y = np.mean([landmarks.part(p).y for p in points])
        return (x, y)

    def detect_body_movement(self, frame):
        """Detecta movimento corporal."""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if hasattr(self, 'previous_frame'):
            diff = cv2.absdiff(self.previous_frame, gray)
            movement = np.sum(diff) / (frame.shape[0] * frame.shape[1])
        else:
            movement = 0
        self.previous_frame = gray
        return movement

    def generate_report(self):
        """Gera o relatório com base nos dados coletados."""
        print("\n--- Relatório de Avaliação ---")
        print(f"Piscos Detectados: {self.blink_counter}")
        print(f"Média de Movimento Corporal: {np.mean(self.body_movements):.2f}")

        if np.mean(self.body_movements) > 0.02:
            print("- Hiperatividade: Movimentos corporais elevados detectados.")

        print(f"Expressões Emocionais: {set(self.emotions)}")
        if "Euforia" in self.emotions and "Tristeza" in self.emotions:
            print("- Bipolaridade: Flutuações emocionais extremas detectadas.")

        self.plot_eye_movements()

    def plot_eye_movements(self):
        """Gera um gráfico dos movimentos oculares."""
        eye_x = [pos[0] for pos in self.eye_movements]
        eye_y = [pos[1] for pos in self.eye_movements]

        plt.plot(eye_x, label='Movimento Horizontal')
        plt.plot(eye_y, label='Movimento Vertical')
        plt.title('Movimentos Oculares')
        plt.xlabel('Frame')
        plt.ylabel('Posição')
        plt.legend()
        plt.show()

# Executar a avaliação
toolkit = NeuroBehaviorToolkit()
toolkit.start_assessment(duration_seconds=30)


In [ ]:
pip install opencv-python dlib numpy tensorflow tensorflow-probability cma imutils matplotlib
